In [1]:
# Install required libraries
!pip install transformers
!pip install PyMuPDF
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 47.4 MB/s eta 0:00:00


In [2]:
# Import libraries
import fitz  # PyMuPDF
from transformers import pipeline
import gradio as gr
from google.colab import files

In [3]:
# Upload PDF
uploaded = files.upload()

Saving Gukesh.pdf to Gukesh.pdf


In [4]:
# Read uploaded file (assuming only one)
filename = next(iter(uploaded))
doc = fitz.open(filename)


In [5]:
# Extract full text from all pages
pdf_text = ""
for page in doc:
    pdf_text += page.get_text()

In [6]:
# Initialize summarizer model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [7]:
# 🔧 Custom min/max length
min_len = 50
max_len = 200

In [8]:
# Summarize (chunked to handle long text)
chunks = [pdf_text[i:i+1000] for i in range(0, len(pdf_text), 1000)]
summary = ""

for chunk in chunks:
    result = summarizer(chunk, max_length=max_len, min_length=min_len, do_sample=False)
    summary += result[0]['summary_text'] + "\n"


In [9]:
# Print and save summary
print("\n📄 Summary:\n", summary)

with open("summary.txt", "w") as f:
    f.write(summary)

files.download("summary.txt")


📄 Summary:
 Gukesh Dommaraju, also known as Gukesh D, is an Indian chess grandmaster. He is the current World Chess Champion. He was introduced to chess at the age of seven, relatively late compared to many other prodigies. His parents, Dr. Rajinikanth (an ENT surgeon) and Dr. Padma (a microbiologist), played a crucial role in his development.
Gukesh became the youngest undisputed World Chess Champion in history, breaking the record previously held by Garry Kasparov. He won the 2024 FIDE Candidates Tournament, earning the right to challenge Ding Liren for the world title. He played a key role in India's historic gold at the 44th Chess Olympiad.
Gukesh is known for his sharp tactical play and mature decision-making. He has won multiple gold medals at the Asian Youth Chess Championship. His success has been attributed to his dedication, hard work, and the support of his team.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
# Define function for Gradio
def summarize_text(text, min_len, max_len):
    chunks = [text[i:i+1000] for i in range(0, len(text), 1000)]
    summary = ""
    for chunk in chunks:
        result = summarizer(chunk, max_length=max_len, min_length=min_len, do_sample=False)
        summary += result[0]['summary_text'] + "\n"
    return summary

In [11]:
# Launch Gradio interface
gr.Interface(
    fn=summarize_text,
    inputs=[gr.Textbox(label="Input Text"),
            gr.Slider(30, 300, label="Min Length"),
            gr.Slider(60, 512, label="Max Length")],
    outputs="text",
    title="Text Summarizer using LLM (BART)"
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3ef009f4a5e0b5d18f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
